In [1]:
# Preliminaries to setup the notebook.  
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import numerical libraries.  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import StandardScaler

#Librairies
import time
import torch
import random

#Raccourcis utilisé
import torch.nn as nn
import torch.utils.data as data_utils
import pandas as pd

# Importation des données

In [2]:
dfl = pd.read_csv('LeaderboardSet2.csv',sep=';')
dfl=dfl.drop('Odor',axis=1)
List=[]
for i in range(len(dfl)):
    if dfl.iat[i,1] == 'replicate ':
        List.append(i)
for i in range(len(List)):
    dfl = dfl.drop(List[i])        
print("Nombre de ligne repliqué que l'on supprime:", len(List))
del List
dfl=dfl.drop('Replicate',axis=1)
dfl = dfl.dropna()
#dfl

Nombre de ligne repliqué que l'on supprime: 0


In [3]:
dft = pd.read_csv('TrainSet.csv',sep=';')
dft=dft.drop('Odor',axis=1)
List=[]
for i in range(len(dft)):
    if dft.iat[i,1] == 'replicate ':
        List.append(i)
for i in range(len(List)):
    dft = dft.drop(List[i])        
print("Nombre de ligne repliqué que l'on supprime:", len(List))
del List
dft=dft.drop('Replicate',axis=1)
dft = dft.dropna()
#dft

Nombre de ligne repliqué que l'on supprime: 1960


In [4]:
df= pd.concat([dft, dfl])
df = df.sort_values(by = ['Compound Identifier','subject'])
del dft
del dfl

# Numérisation des colonne Intensity et Dilution

In [5]:
#Colonne Intenisty
for i in range(len(df)):
    if df.iat[i,1] == 'low ':
        df.iat[i,1] = 0
    if df.iat[i,1] == 'high ':
        df.iat[i,1] = 1
    if df.iat[i,1] == 'low':
        df.iat[i,1] = 0
    if df.iat[i,1] == 'high':
        df.iat[i,1] = 1  

In [6]:
#Colonne Dilution
for i in range(len(df)):
    if df.iat[i,2] == '1/10':
        df.iat[i,2] = 0
    if df.iat[i,2] == '1/1000':
        df.iat[i,2] = 1
    if df.iat[i,2] == '1/1,000 ':
        df.iat[i,2] = 1
    if df.iat[i,2] == '1/100000':
        df.iat[i,2] = 2
    if df.iat[i,2] == '1/100,000 ':
        df.iat[i,2] = 2 
    if df.iat[i,2] == '1/10,000,000 ':
        df.iat[i,2] = 3

In [7]:
df.to_csv('brouillon.csv',sep=';')
df = pd.read_csv('brouillon.csv',sep=';')
df = df.drop('Unnamed: 0',axis=1)

# Labélisation des Moyennes des Descripteurs Olfactifs

In [8]:
df

,Compound Identifier,Intensity,Dilution,subject,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,FRUIT,FISH,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,0,1,1,7,62.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0
1,126,1,0,1,37,60.0,0.0,72.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,126,0,1,2,55,89.0,0.0,33.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,126,1,0,2,64,71.0,0.0,9.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
4,126,0,1,3,89,68.0,0.0,62.0,0.0,0.0,...,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29552,16220109,1,0,47,97,40.0,0.0,0.0,0.0,0.0,...,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0
29553,16220109,0,1,48,5,90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
29554,16220109,1,0,48,94,33.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,7.0
29555,16220109,0,1,49,32,47.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
Liste_High = []
Liste_Low = []
for i in range(len(df)):
    if df.iat[i,1] != 1:
        Liste_High.append(i)
    else:
        Liste_Low.append(i)
Low = df
High = df
        
for i in Liste_High:
    High = High.drop(i)

for i in Liste_Low:
    Low = Low.drop(i)
    
High.to_csv('brouillon.csv',sep=';')
High = pd.read_csv('brouillon.csv',sep=';')
High = High.drop('Unnamed: 0',axis=1)
Low.to_csv('brouillon.csv',sep=';')
Low = pd.read_csv('brouillon.csv',sep=';')
Low = Low.drop('Unnamed: 0',axis=1)
CID = []
for i in range (len(df)):
    CID.append(df.iat[i,0])
CID= list(set(CID))
CID.sort()
print('On fait 2 moyennes pour les', len(CID),
      "molécules, une avec les concentration 'Low', l'autre avec les concentrations 'High'.")

On fait 2 moyennes pour les 407 molécules, une avec les concentration 'Low', l'autre avec les concentrations 'High'.


In [10]:
colo = df.columns
High2 = pd.DataFrame(columns = colo)
for i in range(len(CID)):
    b = High.iloc[i:i+1]
    High2 = pd.concat([High2, b])
    High2.iat[i,0] = CID[i]
    High2.iat[i,1] = 1
    for z in range(2,25):
        High2.iat[i,z] = 0
    cpt = 0
    for j in range(len(High)):
        if High.iat[j,0] == CID[i]:
            cpt +=1
            for z in range(2,25):
                High2.iat[i,z] += High.iat[j,z]
    for z in range(2,25):
        High2.iat[i,z] /= cpt    
High2 = High2.drop('subject',axis=1)
High2 = High2.drop('Dilution',axis=1)

colo = df.columns
Low2 = pd.DataFrame(columns = colo)
for i in range(len(CID)):
    b = Low.iloc[i:i+1]
    Low2 = pd.concat([Low2, b])
    Low2.iat[i,0] = CID[i]
    Low2.iat[i,1] = 0
    for z in range(2,25):
        Low2.iat[i,z] = 0
    cpt = 0
    for j in range(len(Low)):
        if Low.iat[j,0] == CID[i]:
            cpt +=1
            for z in range(2,25):
                Low2.iat[i,z] += Low.iat[j,z]
    for z in range(2,25):
        Low2.iat[i,z] /= cpt    
Low2 = Low2.drop('subject',axis=1)
Low2 = Low2.drop('Dilution',axis=1)

df= pd.concat([Low2, High2])
df = df.sort_values(by = ['Compound Identifier'])
df

,Compound Identifier,Intensity,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,FRUIT,FISH,GARLIC,SPICES,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,0,35.5294,51.058824,0.411765,16.676471,8.147059,0.000000,2.529412,4.970588,...,6.264706,6.264706,5.558824,1.000000,4.029412,3.764706,0.235294,1.794118,5.088235,14.529412
0,126,1,52.7826,48.956522,0.630435,24.347826,6.239130,0.282609,1.608696,3.652174,...,2.847826,0.913043,8.304348,1.500000,1.804348,4.978261,1.260870,1.347826,9.260870,14.239130
1,176,0,13.9375,47.250000,0.562500,2.000000,0.750000,4.750000,3.562500,4.000000,...,2.375000,6.750000,7.312500,7.375000,4.062500,3.937500,4.437500,5.125000,5.687500,14.437500
1,176,1,21.7692,48.461538,2.538462,6.692308,1.230769,0.000000,7.192308,8.230769,...,3.500000,4.230769,6.769231,7.153846,3.615385,3.038462,1.807692,1.961538,4.115385,3.384615
2,177,0,20.1429,53.214286,8.035714,15.000000,3.857143,1.500000,1.607143,1.750000,...,2.071429,2.142857,7.571429,1.178571,0.357143,3.500000,0.142857,1.357143,5.214286,7.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,6429333,0,22.4706,57.588235,4.529412,9.470588,2.117647,0.000000,6.647059,4.411765,...,0.000000,9.764706,4.470588,1.647059,3.941176,0.058824,1.000000,0.294118,5.588235,12.588235
405,6999977,1,52.6667,51.190476,13.666667,18.285714,5.500000,0.261905,3.642857,4.261905,...,1.000000,3.571429,3.428571,3.404762,0.619048,3.166667,4.547619,2.071429,2.738095,10.119048
405,6999977,0,41.619,45.285714,3.904762,11.523810,9.428571,0.000000,5.333333,2.190476,...,2.333333,10.809524,12.666667,5.095238,2.571429,10.666667,5.047619,3.000000,6.190476,12.761905
406,16220109,0,47.2439,47.951220,0.780488,6.146341,3.707317,1.097561,0.292683,6.073171,...,7.073171,5.707317,3.317073,2.292683,1.463415,3.902439,7.707317,7.317073,7.219512,11.878049


In [11]:
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')
df = pd.read_csv('Senteur Dataset Dream 2.csv',sep=';')
df = df.drop('Unnamed: 0',axis=1)
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')

In [12]:
#Colonnes sentation Olfactive
for i in range(len(df)):
    for j in range (2,23):
        if df.iat[i,j] == 0:
            df.iat[i,j] = 0
        elif df.iat[i,j] == 100:
            df.iat[i,j] = 10
        else :
            df.iat[i,j] = (df.iat[i,j] // 10) +1

In [13]:
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')
df = pd.read_csv('Senteur Dataset Dream 2.csv',sep=';')
df = df.drop('Unnamed: 0',axis=1)
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')

# Valeurs moléculaire

In [14]:
mdf = pd.read_csv('molecular_descriptors_data2.csv',sep=';')
mdf = mdf.dropna()
print(" On drop les Molécules 807, 887, 962, 6505, et 8122.",
      "La 962 est présente dans le dataframe test.")
print("Ces molécules présente de nombreux NaN dans leurs données moléculaires.")

mdf.to_csv('Mol_descriptor.csv',sep=';')
mdf = pd.read_csv('Mol_descriptor.csv',sep=';')
mdf = mdf.drop('Unnamed: 0',axis=1)
mdf.to_csv('Mol_descriptor.csv',sep=';')

print("On supprime les données olfactives liées à ces molécules:")
Cmdf = pd.DataFrame(mdf[['CID']],columns = ['CID'])
List= []
for i in range (len(df)):
    a=0
    for j in range (len(Cmdf)):
        if Cmdf.iat[j,0] == df.iat[i,0]:
            a+=1
    if a == 0:
        List.append(i)
print('Nombre de ligne concernée:', len(List))
for i in range(len(List)):
    df = df.drop(List[i])
    
Liste=[]
Xcolumns= mdf.columns
for j in range (len(Xcolumns)):
    a=0
    for i in range (len(mdf)):
        if mdf.iat[i,j] != 0 :
            a+=1
    if a == 0:
        Liste.append(j)
print('Nombre de colonne sans importance:',len(Liste))
print('On les supprime.')
for j in range(len(Liste)):
    mdf.drop([Xcolumns[Liste[j]]], axis = 1, inplace = True)  
del Liste
del Cmdf

 On drop les Molécules 807, 887, 962, 6505, et 8122. La 962 est présente dans le dataframe test.
Ces molécules présente de nombreux NaN dans leurs données moléculaires.
On supprime les données olfactives liées à ces molécules:
Nombre de ligne concernée: 8
Nombre de colonne sans importance: 1790
On les supprime.


In [15]:
#Pour ne pas modifier la colonne CID, on navigue sur les colonnes de 1 à 3117 
#en ajoutant +1 à j qui va de 0 à 3116.
#Code pour la racine cubique, permet une meilleure normalisation

for j in range (3079):
    for i in range(len(mdf)):
        if mdf.iat[i,j+1] < 0:
            mdf.iat[i,j+1] = ((mdf.iat[i,j+1]**(2))**(1/2))**(1/3)
            mdf.iat[i,j+1] = (mdf.iat[i,j+1])*1000//1/1000
            mdf.iat[i,j+1] *= (-1)
        else :
            mdf.iat[i,j+1] = ((mdf.iat[i,j+1]**(2))**(1/2))**(1/3)
            mdf.iat[i,j+1] = (mdf.iat[i,j+1])*1000//1/1000

### Alignement des index des deux dataframes

In [16]:
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')
df = pd.read_csv('Senteur Dataset Dream 2.csv',sep=';')
df = df.drop('Unnamed: 0',axis=1)
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')

In [17]:
mdf.to_csv('Mol_descriptor.csv',sep=';')
mdf = pd.read_csv('Mol_descriptor.csv',sep=';')
mdf = mdf.drop('Unnamed: 0',axis=1)
mdf.to_csv('Mol_descriptor.csv',sep=';')

In [ ]:
mdf

In [18]:
df

,Compound Identifier,Intensity,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,FRUIT,FISH,GARLIC,SPICES,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,0,4.0,6.0,1.0,2.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
1,126,1,6.0,5.0,1.0,3.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
2,176,0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
3,176,1,3.0,5.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,177,0,3.0,6.0,1.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,6429333,0,3.0,6.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
802,6999977,1,6.0,6.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
803,6999977,0,5.0,5.0,1.0,2.0,1.0,0.0,1.0,1.0,...,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0
804,16220109,0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


## Création du Dataset Train

Le but de ce dataset est d'aligner autant de lignes de descripteurs moléculaires qu'il y a de ligne de sensation olfactive perçus. Soit 30812.  
On traite la création de ces données molécules par molécules.

In [19]:
Cdf = df[['Compound Identifier']]
colo = mdf.columns
tdf= pd.DataFrame(columns = colo)
for j in range(len(mdf)):
    inter = pd.DataFrame(columns = colo)
    for i in range(len(Cdf)):
        if mdf.iat[j,0] == Cdf.iat[i,0] :
            b= mdf.iloc[j:j+1]
            inter = pd.concat([inter, b])
    tdf = pd.concat([tdf, inter])
    print('Chargement:',j+1,'/471;          Molécule:',mdf.iat[j,0],'              Nombre de ligne:',len(inter))
    
del b
del inter
del Cdf
del colo

Chargement: 1 /471;          Molécule: 126               Nombre de ligne: 2
Chargement: 2 /471;          Molécule: 176               Nombre de ligne: 2
Chargement: 3 /471;          Molécule: 177               Nombre de ligne: 2
Chargement: 4 /471;          Molécule: 180               Nombre de ligne: 0
Chargement: 5 /471;          Molécule: 196               Nombre de ligne: 2
Chargement: 6 /471;          Molécule: 239               Nombre de ligne: 2
Chargement: 7 /471;          Molécule: 240               Nombre de ligne: 2
Chargement: 8 /471;          Molécule: 241               Nombre de ligne: 2
Chargement: 9 /471;          Molécule: 243               Nombre de ligne: 2
Chargement: 10 /471;          Molécule: 244               Nombre de ligne: 2
Chargement: 11 /471;          Molécule: 261               Nombre de ligne: 2
Chargement: 12 /471;          Molécule: 262               Nombre de ligne: 0
Chargement: 13 /471;          Molécule: 263               Nombre de ligne: 2
Chargeme

Chargement: 107 /471;          Molécule: 7150               Nombre de ligne: 2
Chargement: 108 /471;          Molécule: 7151               Nombre de ligne: 2
Chargement: 109 /471;          Molécule: 7165               Nombre de ligne: 2
Chargement: 110 /471;          Molécule: 7194               Nombre de ligne: 2
Chargement: 111 /471;          Molécule: 7288               Nombre de ligne: 2
Chargement: 112 /471;          Molécule: 7302               Nombre de ligne: 0
Chargement: 113 /471;          Molécule: 7335               Nombre de ligne: 2
Chargement: 114 /471;          Molécule: 7341               Nombre de ligne: 2
Chargement: 115 /471;          Molécule: 7360               Nombre de ligne: 2
Chargement: 116 /471;          Molécule: 7361               Nombre de ligne: 2
Chargement: 117 /471;          Molécule: 7409               Nombre de ligne: 2
Chargement: 118 /471;          Molécule: 7410               Nombre de ligne: 2
Chargement: 119 /471;          Molécule: 7463       

Chargement: 211 /471;          Molécule: 8918               Nombre de ligne: 2
Chargement: 212 /471;          Molécule: 9012               Nombre de ligne: 0
Chargement: 213 /471;          Molécule: 9016               Nombre de ligne: 2
Chargement: 214 /471;          Molécule: 9024               Nombre de ligne: 2
Chargement: 215 /471;          Molécule: 9025               Nombre de ligne: 2
Chargement: 216 /471;          Molécule: 9256               Nombre de ligne: 2
Chargement: 217 /471;          Molécule: 9261               Nombre de ligne: 2
Chargement: 218 /471;          Molécule: 9589               Nombre de ligne: 2
Chargement: 219 /471;          Molécule: 9609               Nombre de ligne: 2
Chargement: 220 /471;          Molécule: 9862               Nombre de ligne: 2
Chargement: 221 /471;          Molécule: 10285               Nombre de ligne: 2
Chargement: 222 /471;          Molécule: 10364               Nombre de ligne: 2
Chargement: 223 /471;          Molécule: 10400    

Chargement: 315 /471;          Molécule: 27458               Nombre de ligne: 2
Chargement: 316 /471;          Molécule: 31209               Nombre de ligne: 2
Chargement: 317 /471;          Molécule: 31210               Nombre de ligne: 2
Chargement: 318 /471;          Molécule: 31219               Nombre de ligne: 0
Chargement: 319 /471;          Molécule: 31225               Nombre de ligne: 2
Chargement: 320 /471;          Molécule: 31234               Nombre de ligne: 2
Chargement: 321 /471;          Molécule: 31244               Nombre de ligne: 2
Chargement: 322 /471;          Molécule: 31246               Nombre de ligne: 2
Chargement: 323 /471;          Molécule: 31249               Nombre de ligne: 2
Chargement: 324 /471;          Molécule: 31252               Nombre de ligne: 2
Chargement: 325 /471;          Molécule: 31260               Nombre de ligne: 2
Chargement: 326 /471;          Molécule: 31265               Nombre de ligne: 2
Chargement: 327 /471;          Molécule:

Chargement: 419 /471;          Molécule: 521238               Nombre de ligne: 2
Chargement: 420 /471;          Molécule: 526618               Nombre de ligne: 2
Chargement: 421 /471;          Molécule: 556940               Nombre de ligne: 2
Chargement: 422 /471;          Molécule: 565690               Nombre de ligne: 2
Chargement: 423 /471;          Molécule: 595928               Nombre de ligne: 2
Chargement: 424 /471;          Molécule: 637563               Nombre de ligne: 2
Chargement: 425 /471;          Molécule: 637566               Nombre de ligne: 2
Chargement: 426 /471;          Molécule: 637758               Nombre de ligne: 0
Chargement: 427 /471;          Molécule: 637776               Nombre de ligne: 2
Chargement: 428 /471;          Molécule: 637796               Nombre de ligne: 2
Chargement: 429 /471;          Molécule: 638014               Nombre de ligne: 2
Chargement: 430 /471;          Molécule: 638024               Nombre de ligne: 2
Chargement: 431 /471;       

In [20]:
tdf

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Depressant-80,Depressant-50,Hypertens-80,Hypertens-50,Hypnotic-80,Hypnotic-50,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,126,4.532,4.961,2.011,2.155,2.482,2.168,2.554,0.873,1.006,...,0,0,0,0,0,0,0,0,0,0
0,126,4.532,4.961,2.011,2.155,2.482,2.168,2.554,0.873,1.006,...,0,0,0,0,0,0,0,0,0,0
1,176,3.141,3.916,1.958,1.648,2.034,1.642,2.099,0.824,1.017,...,0,0,0,0,0,0,0,0,0,0
1,176,3.141,3.916,1.958,1.648,2.034,1.642,2.099,0.824,1.017,...,0,0,0,0,0,0,0,0,0,0
2,177,2.175,3.531,1.846,1.556,1.921,1.584,2.003,0.813,1.004,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,6429333,5.336,5.268,1.909,2.372,2.747,2.425,2.855,0.859,0.995,...,0,0,0,0,0,0,0,0,1,0
468,6999977,4.657,5.268,1.826,2.341,2.891,2.392,3.019,0.811,1.002,...,0,0,0,0,0,0,0,0,0,0
468,6999977,4.657,5.268,1.826,2.341,2.891,2.392,3.019,0.811,1.002,...,0,0,0,0,0,0,0,0,0,0
470,16220109,5.867,5.670,1.804,2.579,3.128,2.657,3.275,0.821,0.995,...,0,0,0,0,1,0,1,0,1,0


In [21]:
df.to_csv('Senteur Dataset Dream 2.csv',sep=';')
mdf.to_csv('Mol_descriptor.csv',sep=';')
tdf.to_csv('brouillon.csv',sep=';')
tdf = pd.read_csv('brouillon.csv',sep=';')
tdf = tdf.drop('Unnamed: 0',axis=1)
tdf.to_csv('Molecular Dataset Dream 2.csv',sep=';')

## Création du Dataset Test

68 Molécules, la 962 possédant des Nan a été retiré.

#### Enregistrement des Dataset crées